In [0]:
pip install azure-storage-blob

Python interpreter will be restarted.
  Using cached azure_storage_blob-12.16.0-py3-none-any.whl (387 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_core-1.27.1-py3-none-any.whl (174 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b598bf91-0ac5-4b63-b46f-679c46a835fb
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install wordcloud

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
!pip install pandas_bokeh
import pandas_bokeh


     |███████

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 7.8 MB 45.0 MB/s 
     |████████████████████████████████| 300 kB 63.2 MB/s 
     |████████████████████████████████| 661 kB 56.3 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b598bf91-0ac5-4b63-b46f-679c46a835fb/bin/python -m pip install --upgrade pip' command.


In [0]:
from ipywidgets import interact
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [0]:
import requests
import pandas as pd
import json
import plotly
import plotly.graph_objects as go
import gzip
import matplotlib.pyplot as plt
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, explode,to_date, date_format, count, desc, countDistinct, regexp_extract,regexp_replace
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, ArrayType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from wordcloud import WordCloud
from nltk.corpus import stopwords
from pyspark.sql.functions import dayofmonth, countDistinct, to_date, hour
from pyspark.sql.functions import explode, lower, length, trim, regexp_replace

In [0]:
spark = SparkSession.builder.getOrCreate()

storage_account_name = "pod4projectstorage"
storage_account_access_key = "2hClDrVPLGX4QBDBk8OylAkHqczIQfDja66Yl488rmj/0+vb+CAzOxL5qMe5XyM9ZupgwveVRm3N+AStriO5vg=="

container_name = "data"
directory_path = ""

spark.conf.set("fs.azure.account.key." + storage_account_name + ".blob.core.windows.net", storage_account_access_key)

df = spark.read.parquet("wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/" + directory_path)


In [0]:
push_df = df.filter(df.type == "PushEvent")

In [0]:
for col_name in push_df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349, ihsan...|  null|ea8e2f6ddc2b0b90b...|[{{ihsannurul@hot...|    622742349|  null| null|13478051933|       null|   refs/heads/master|   1|    null|
|PushEvent|  true|2023-05-01T00:00:00Z|         

In [0]:
push_df = push_df.withColumn("commit", explode("commits")).withColumn("commit_author", col("commit.author.name")).withColumn("commit_message", col("commit.message")).withColumn("login", col("actor.login")).withColumn("url", col("commit.url")).withColumn("day_of_week", date_format(to_date(col("created_at"), "yyyy-MM-dd'T'HH:mm:ss'Z'"), "EEEE")).withColumn("domain", regexp_extract(col("url"), r".*repos/(.*)/commits/.*", 1)).drop("commits", "commit","url")

In [0]:
count_by_day = push_df.groupBy("day_of_week").count().orderBy(desc("count"))
count_by_day.show()

+-----------+--------+
|day_of_week|   count|
+-----------+--------+
|  Wednesday|17094241|
|    Tuesday|16726665|
|     Monday|16537903|
|   Thursday|14070302|
|     Friday|13783353|
|     Sunday|11023825|
|   Saturday|10884481|
+-----------+--------+



In [0]:
count_pd = count_by_day.toPandas()

fig1 = go.Figure(data=go.Bar(x=count_pd["day_of_week"], y=count_pd["count"],marker=dict(color='orangered')))
fig1.update_traces( marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6,width=0.7)
fig1.update_layout(
    title="Number of Commits by Day of week", xaxis_title="Day of the Week", yaxis_title="Count", width =1000, height=550)

fig1.show()

In [0]:
commit_msg_by_author_counts = push_df.groupBy("commit_message").agg(countDistinct(col("commit_author")).alias("author_count")).orderBy(desc("author_count"))

In [0]:
commit_msg_by_author_counts.show()

+--------------------+------------+
|      commit_message|author_count|
+--------------------+------------+
|    Update README.md|      483535|
|Add files via upload|      424677|
|    Create README.md|      145631|
|   Update index.html|       82781|
|              update|       68786|
|                test|       38987|
|        first commit|       37281|
|        Create CNAME|       32629|
|                 fix|       30865|
|    Delete README.md|       27643|
|      Initial commit|       27084|
|             Updates|       24410|
|        Delete CNAME|       24186|
|      initial commit|       22223|
|                   .|       21493|
|              commit|       19131|
|   Delete index.html|       19128|
|              Update|       17724|
|   Create index.html|       17059|
|    Update style.css|       15654|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_messages = commit_msg_by_author_counts.limit(10).toPandas()
fig2 = go.Figure(data=go.Bar(x=top_commit_messages['commit_message'], y=top_commit_messages['author_count'], marker =dict(color='orangered')))
fig2.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig2.update_layout(title='Top 10 Commit Messages by Author Count', xaxis_title='Commit Message', yaxis_title='Author Count',width = 1000, height=550)
fig2.show()

In [0]:
commit_by_domain_counts = push_df.groupBy("domain").agg(count("*").alias("commit_count")).orderBy(desc("commit_count"))

In [0]:
commit_by_domain_counts.show()

+--------------------+------------+
|              domain|commit_count|
+--------------------+------------+
|mizzled/comitigation|     4079680|
|pythoneerHiro/__e...|      659391|
|hqshit/i-hate-summer|      409384|
|   rmayr/rediffusion|      284868|
|  happyfish2024/mins|      281248|
|WolseyBankWitness...|      235772|
|bullet-dev-team/p...|      232328|
|     mo9a7i/time_now|      206438|
|   Nasfame/__garbage|      163920|
|networkoperator/d...|      161908|
|Christoffel-T/fiv...|      151895|
|jeyserHiro/__garbage|      148620|
|pythoneerHiro/__g...|      136420|
|bullet-dev-team/d...|      118041|
|DF-Master/DF-Mast...|      116782|
|Hardwaregore/gimm...|      115728|
|pythoneerHiro/__t...|      107106|
|    vpnsuperapp/fast|       99840|
|    Zebrainy/upptime|       99340|
|splashingdiamonds...|       94264|
+--------------------+------------+
only showing top 20 rows



In [0]:
top_commit_domains = commit_by_domain_counts.limit(10).toPandas()
fig3 = go.Figure(data=go.Bar(x=top_commit_domains['domain'], y=top_commit_domains['commit_count'],marker=dict(color='orangered')))
fig3.update_traces( marker_line_color='rgb(8,48,107)', marker_line_width=1.5, opacity=0.6,width=0.7)
fig3.update_layout(title='Top 10 Commit domains', xaxis_title='domains', yaxis_title='Count', width = 1000, height=550)
fig3.show()

In [0]:
issues_df = df.filter(df.type=='PushEvent')

In [0]:
for col_name in issues_df.columns:
    new_col_name = col_name.replace("payload.", "")
    issues_df = issues_df.withColumnRenamed(col_name, new_col_name)
issues_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349, ihsan...|  null|ea8e2f6ddc2b0b90b...|[{{ihsannurul@hot...|    622742349|  null| null|13478051933|       null|   refs/heads/master|   1|    null|
|PushEvent|  true|2023-05-01T00:00:00Z|         

In [0]:
descrip = issues_df.select(explode('commits.message').alias("message"))

In [0]:
grouped_df = df.groupBy("type").count().orderBy("count", ascending=False)

In [0]:
pd_data = grouped_df.toPandas()
pd_data

,type,count
0,PushEvent,62375369
1,CreateEvent,15508107
2,PullRequestEvent,8610684
3,WatchEvent,6525132
4,IssueCommentEvent,5170142
5,DeleteEvent,3540874
6,PullRequestReviewEvent,2834574
7,IssuesEvent,2366505
8,PullRequestReviewCommentEvent,1685195
9,ForkEvent,1555745


In [0]:
fig4 = go.Figure([go.Bar(x=pd_data['type'], y=pd_data['count'],marker=dict(color='orangered'))])
fig4.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6)
fig4.update_layout(title_text='Event Counts',xaxis=dict(tickangle=90), width =1000, height=550)
fig4.show()

In [0]:
push_df = issues_df

In [0]:
for col_name in df.columns:
    new_col_name = col_name.replace("payload.", "")
    push_df = push_df.withColumnRenamed(col_name, new_col_name)
push_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349, ihsan...|  null|ea8e2f6ddc2b0b90b...|[{{ihsannurul@hot...|    622742349|  null| null|13478051933|       null|   refs/heads/master|   1|    null|
|PushEvent|  true|2023-05-01T00:00:00Z|         

In [0]:
hour_df = push_df.withColumn('hour', hour(push_df['created_at']))

In [0]:
hour_df.show()

+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+----+
|     type|public|          created_at|                 org|               actor|                repo|action|              before|             commits|repository_id|review|pages|    push_id|pusher_type|                 ref|size|ref_type|hour|
+---------+------+--------------------+--------------------+--------------------+--------------------+------+--------------------+--------------------+-------------+------+-----+-----------+-----------+--------------------+----+--------+----+
|PushEvent|  true|2023-05-01T00:00:00Z|                null|{https://avatars....|{622742349, ihsan...|  null|ea8e2f6ddc2b0b90b...|[{{ihsannurul@hot...|    622742349|  null| null|13478051933|       null|   refs/heads/master|   1|    null|   0|
|PushEvent|  true|2023-05-01

In [0]:
grouped_hour = hour_df.groupBy("hour").count().orderBy("count", ascending=False)

In [0]:
pd_hour = grouped_hour.toPandas()

In [0]:
fig5 = go.Figure([go.Bar(x=pd_hour['hour'], y=pd_hour['count'],marker=dict(color='orangered'))])
fig5.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig5.update_layout(title_text='Commits by Hour',width = 1000, height = 550)
fig5.show()

In [0]:
result = push_df.groupBy(dayofmonth("created_at").alias("Day")).agg(countDistinct("actor.id").alias("Users")).orderBy("Users", ascending=False)

In [0]:
result.show()

+---+------+
|Day| Users|
+---+------+
| 16|424008|
| 24|423598|
| 23|422278|
| 10|418778|
|  4|416232|
| 31|414916|
|  9|413888|
| 11|413709|
| 17|413524|
| 25|413127|
| 30|412549|
|  3|411701|
| 22|409983|
| 15|408045|
|  2|407199|
|  8|405626|
| 18|403107|
|  5|391737|
| 26|390119|
| 12|390003|
+---+------+
only showing top 20 rows



In [0]:
pd_date = result.toPandas()

In [0]:
fig6 = go.Figure([go.Bar(x=pd_date['Day'], y=pd_date['Users'], marker=dict(
        color=['yellowgreen' if val == pd_date['Users'].max() else 'orangered' for val in pd_date['Users']]))])
fig6.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.7)
fig6.update_layout(title_text='Commits by Day of the Month',width = 1000, height = 550)
fig6.show()

In [0]:
grouped_pushdf = push_df.select('org.login').dropna()

In [0]:
grouped_pushdf = grouped_pushdf.groupBy("login").count().orderBy("count", ascending=False)

In [0]:
grouped_pushdf.show()

+--------------------+------+
|               login| count|
+--------------------+------+
|          tf-vcs-e2e|188786|
|     networkoperator|161876|
|           newstools| 86995|
|         isomerpages| 74550|
|            Risk-DAO| 73382|
| redhat-appstudio-qe| 69836|
|        magicwave-io| 68005|
|         feedarchive| 54788|
|Ultimate-Hosts-Bl...| 46156|
|  MetadataGitTesting| 45595|
|            acciojob| 37684|
|redhat-appstudio-...| 36511|
|           microsoft| 31454|
|               hmcts| 31333|
|             unifyai| 30700|
|                odoo| 30568|
|     ecomplus-stores| 29155|
|    scalr-automation| 25363|
|           rust-lang| 25294|
|          factcheckr| 23613|
+--------------------+------+
only showing top 20 rows



In [0]:
pd_pushdf = grouped_pushdf.toPandas()
pd_pushdf = pd_pushdf.head()

In [0]:
pd_pushdf

,login,count
0,tf-vcs-e2e,188786
1,networkoperator,161876
2,newstools,86995
3,isomerpages,74550
4,Risk-DAO,73382


In [0]:
fig7 = go.Figure([go.Bar(x=pd_pushdf['login'], y=pd_pushdf['count'],marker=dict(color='orangered'))])
fig7.update_traces( marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.6,width=0.5)
fig7.update_layout(title_text='Organizations with highest number of commits',width = 1000, height = 550)
fig7.show()

In [0]:
exploded_msg = push_df.select(explode('commits.message').alias("message"))

In [0]:
exploded_msg.dropna()
exploded_msg.show()

+--------------------+
|             message|
+--------------------+
|Parametro cada cu...|
|       Delete Client|
|              deploy|
|         QIAAEngines|
|Updated with Dev ...|
|Commit 403 of 100...|
|    Update README.md|
|deploy: 76c7cbb02...|
|     ex04 incomplete|
|creating and list...|
|               task6|
|add responsive st...|
|Add files via upload|
|Latest data: Wed ...|
|Update from Githu...|
|removed lazy load...|
|Generate developm...|
|2023.05.04 07:59:...|
|       change colour|
|Merge pull reques...|
+--------------------+
only showing top 20 rows



In [0]:
pandas_bokeh.plot_grid([[fig1.show(), fig2.show()], [fig3.show(), fig4.show()], [fig5.show(), fig6.show(), fig7.show()]], width=200, height=200)
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [0]:
exploded_msg = exploded_msg.withColumn("message", trim(lower(regexp_replace(exploded_msg.message, "[^a-zA-Z\\s]", ""))))

In [0]:
exploded_msg = exploded_msg.filter(length(exploded_msg.message) > 0)


In [0]:
exploded_msg.show()

+--------------------+
|             message|
+--------------------+
|parametro cada cu...|
|       delete client|
|              deploy|
|         qiaaengines|
|updated with dev ...|
|          commit  of|
|     update readmemd|
|deploy ccbbedddaa...|
|       ex incomplete|
|creating and list...|
|                task|
|add responsive st...|
|add files via upload|
|latest data wed m...|
|update from githu...|
|removed lazy load...|
|generate developm...|
|       change colour|
|merge pull reques...|
|deploy to github ...|
+--------------------+
only showing top 20 rows



In [0]:
tokenizer = Tokenizer(inputCol="message", outputCol="tokens")
tz_msg = tokenizer.transform(exploded_msg)

In [0]:
tz_msg.show()

+--------------------+--------------------+
|             message|              tokens|
+--------------------+--------------------+
|parametro cada cu...|[parametro, cada,...|
|       delete client|    [delete, client]|
|              deploy|            [deploy]|
|         qiaaengines|       [qiaaengines]|
|updated with dev ...|[updated, with, d...|
|          commit  of|      [commit, , of]|
|     update readmemd|  [update, readmemd]|
|deploy ccbbedddaa...|[deploy, ccbbeddd...|
|       ex incomplete|    [ex, incomplete]|
|creating and list...|[creating, and, l...|
|                task|              [task]|
|add responsive st...|[add, responsive,...|
|add files via upload|[add, files, via,...|
|latest data wed m...|[latest, data, we...|
|update from githu...|[update, from, gi...|
|removed lazy load...|[removed, lazy, l...|
|generate developm...|[generate, develo...|
|       change colour|    [change, colour]|
|merge pull reques...|[merge, pull, req...|
|deploy to github ...|[deploy, t

In [0]:
remover = StopWordsRemover(inputCol='tokens', outputCol='fil_msg')
tz_msg = remover.transform(tz_msg).drop('tokens')

In [0]:
tz_msg.show()

+--------------------+--------------------+
|             message|             fil_msg|
+--------------------+--------------------+
|parametro cada cu...|[parametro, cada,...|
|       delete client|    [delete, client]|
|              deploy|            [deploy]|
|         qiaaengines|       [qiaaengines]|
|updated with dev ...|[updated, dev, me...|
|          commit  of|          [commit, ]|
|     update readmemd|  [update, readmemd]|
|deploy ccbbedddaa...|[deploy, ccbbeddd...|
|       ex incomplete|    [ex, incomplete]|
|creating and list...|[creating, listin...|
|                task|              [task]|
|add responsive st...|[add, responsive,...|
|add files via upload|[add, files, via,...|
|latest data wed m...|[latest, data, we...|
|update from githu...|[update, github, ...|
|removed lazy load...|[removed, lazy, l...|
|generate developm...|[generate, develo...|
|       change colour|    [change, colour]|
|merge pull reques...|[merge, pull, req...|
|deploy to github ...|[deploy, g